In [ ]:
# 필요한 모든 라이브러리 임포트

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import networkx as nx
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.image as mpimg
import random

In [ ]:
# H&M 데이터셋 로드

articles = pd.read_csv("../your data path here/articles.csv")
transactions = pd.read_csv("../your data path here/transactions_train.csv")

In [ ]:
# 아이템과 사용자에 대한 빈도를 계산합니다.
item_freq = transactions.groupby('article_id')['customer_id'].nunique() #아이템별로 몇 명의 고객이 구매했는지
user_freq = transactions.groupby('customer_id')['article_id'].nunique() #사용자별로 몇 개의 아이템을 구매했는지

# 빈도가 100회 이상인 아이템과 사용자만 선택합니다.
items = item_freq[item_freq >= 100].index
users = user_freq[user_freq >= 100].index

# 선택된 아이템과 사용자에 해당하는 행만 남깁니다.
filtered_df = transactions[transactions['article_id'].isin(items) & transactions['customer_id'].isin(users)]

In [ ]:
# weight 추가를 위한 집계 
# 'customer_id'와 'article_id' 쌍별로 등장하는 빈도를 계산합니다.
# 각 고객이 각 아이템을 얼마나 많이 구매했는지 (유저의 아이템별 재구매 횟수)
freq = filtered_df.groupby(['customer_id', 'article_id']).size().reset_index(name='frequency')

# 'frequency' column을 추가합니다.
GraphTravel_HM = filtered_df.merge(freq, on=['customer_id', 'article_id'], how='left')

# 'frequency'가 10 이상인 행만 df에 남깁니다.
GraphTravel_HM = GraphTravel_HM[GraphTravel_HM['frequency'] >= 10]

In [ ]:
display(GraphTravel_HM)

print("unique customer_id" , GraphTravel_HM.customer_id.nunique())
print("unique article_id" , GraphTravel_HM.article_id.nunique())

In [ ]:
# frequency column의 히스토그램 및 KDE 그래프를 그립니다.
sns.distplot(GraphTravel_HM['frequency'], kde=True, bins=30)

plt.title('Distribution of frequency')
plt.xlabel('Frequency')
plt.ylabel('Density')

plt.show()

In [ ]:
# 해시값인 customer_id 정수 매핑
unique_customer_ids = GraphTravel_HM['customer_id'].unique()
customer_id_mapping = {id: i for i, id in enumerate(unique_customer_ids)}
GraphTravel_HM['customer_id'] = GraphTravel_HM['customer_id'].map(customer_id_mapping)

# article_id를 prod_name에 매핑하는 dictionary를 생성
item_name_mapping = dict(zip(articles['article_id'], articles['prod_name'])) # prod_name

In [ ]:
# 그래프 초기화
G = nx.Graph()

# 노드와 엣지를 추가
for index, row in GraphTravel_HM.iterrows():
    G.add_node(row['customer_id'], type='user')
    G.add_node(row['article_id'], type='item')
    
    # 'customer_id'와 'article_id' 사이에 가중치가 'frequency'인 엣지를 추가
    G.add_edge(row['customer_id'], row['article_id'], weight=row['frequency'])

In [ ]:
# biased random walk 코드 
def biased_random_walk(G, start_node, walk_length, p=1, q=1):
    walk = [start_node]

    while len(walk) < walk_length:
        cur_node = walk[-1]
        cur_neighbors = list(G.neighbors(cur_node))

        if len(cur_neighbors) > 0:
            if len(walk) == 1:
                walk.append(random.choice(cur_neighbors))
            else:
                prev_node = walk[-2]

                probability = []
                for neighbor in cur_neighbors:
                    if neighbor == prev_node:
                        # Return parameter : 방문한 적 있는 노드로 돌아갈 확률 제어
                        probability.append(1/p)
                    elif G.has_edge(neighbor, prev_node):
                        # Stay parameter : 타겟 노드의 이웃 노드로 이동할 확률 1
                        probability.append(1)
                    else:
                        # In-out parameter : 타겟 노드의 이웃이 아닌 노드로 이동할 확률 제어
                        probability.append(1/q)

                probability = np.array(probability)
                probability = probability / probability.sum()  # normalize

                next_node = np.random.choice(cur_neighbors, p=probability)
                walk.append(next_node)
        else:
            break

    return walk

In [ ]:
def generate_walks(G, num_walks, walk_length, p=1, q=1):
    walks = []
    nodes = list(G.nodes())
    for _ in range(num_walks):
        random.shuffle(nodes)  # to ensure randomness
        for node in nodes:
            walk_from_node = biased_random_walk(G, node, walk_length, p, q)
            walks.append(walk_from_node)
    return walks

In [ ]:
# Random Walk 생성
walks = generate_walks(G, num_walks=10, walk_length=20, p=9, q=1)

# walks는 random walk 결과를 담은 리스트라고 가정
filtered_walks = [walk for walk in walks if len(walk) >= 5]

# String 형태로 변환 (Word2Vec 입력을 위해)
walks = [[str(node) for node in walk] for walk in walks]

# Word2Vec 학습
model = Word2Vec(walks, vector_size=128, window=5, min_count=0,  hs=1, sg=1, workers=4, epochs=10)

# 노드 임베딩 추출
embeddings = {node_id: model.wv[node_id] for node_id in model.wv.index_to_key}

In [ ]:
def get_user_embedding(user_id, embeddings):
    return embeddings[str(user_id)]

In [ ]:
def get_rated_items(user_id, df):
    return set(df[df['customer_id'] == user_id]['article_id'])

In [ ]:
def calculate_similarities(user_id, df, embeddings):
    rated_items = get_rated_items(user_id, df)
    user_embedding = get_user_embedding(user_id, embeddings)

    item_similarities = []
    for item_id in set(df['article_id']):
        if item_id not in rated_items:  # 이미 구매한 제품은 제외
            item_embedding = embeddings[str(item_id)]
            similarity = cosine_similarity([user_embedding], [item_embedding])[0][0]
            item_similarities.append((item_id, similarity))

    return item_similarities

In [ ]:
def show_images(items, item_name_mapping, num_items, show_similarity=False):
    # True로 설정되면, 아이템의 similarity score도 함께 출력됩니다!
    f, ax = plt.subplots(1, num_items, figsize=(20,10))
    if num_items == 1:
        ax = [ax]
    for i, item in enumerate(items):
        item_id, similarity = item
        print(f"- Item {item_id}: {item_name_mapping[item_id]}", end='')
        if show_similarity:
            print(f" with similarity score: {similarity}")
        else:
            print()
        img_path = f"../input/h-and-m-personalized-fashion-recommendations/images/0{str(item_id)[:2]}/0{int(item_id)}.jpg"
        try:
            img = mpimg.imread(img_path)
            ax[i].imshow(img)
            ax[i].set_title(f'Item {item_id}')
            ax[i].set_xticks([], [])
            ax[i].set_yticks([], [])
            ax[i].grid(False)
        except FileNotFoundError:
            print(f"Image for item {item_id} not found.")
    plt.show()

In [ ]:

def recommend_items(user_id, df, embeddings, item_name_mapping, num_items=5):
    # 사용자가 이미 구매한 제품 찾기
    rated_items = get_rated_items(user_id, df)
    
    # 사용자가 구매한 아이템의 이미지 출력
    print(f"User {user_id} has purchased:")
    show_images([(item_id, 0) for item_id in list(rated_items)[:5]], item_name_mapping, min(len(rated_items), 5))
    
    # 각 아이템 노드에 대한 유사도를 계산
    item_similarities = calculate_similarities(user_id, df, embeddings)

    # 유사도가 가장 높은 아이템들을 추천
    recommended_items = sorted(item_similarities, key=lambda x: x[1], reverse=True)[:num_items]

    print(f"\nRecommended items for user {user_id}:")
    show_images(recommended_items, item_name_mapping, num_items, show_similarity=True)


In [ ]:
# 예제: 유저 45에 대한 추천 상위 5개 
recommend_items(45, GraphTravel_HM, embeddings, item_name_mapping, num_items=5)